# Credential Issuance and Presentation

<div class="alert alert-primary">
<b>🎯 OBJECTIVE</b><hr>
Demonstrate the process of issuing an Authentic Chained Data Container (ACDC), also known as a Verifiable Credential (VC), from an Issuer to a Holder using the Issuance and Presentation Exchange (IPEX) protocol. This involves setting up identities, defining a credential schema, creating the credential, and securely transferring it.
</div> 

In [1]:
import { randomPasscode, Serder} from 'npm:signify-ts';
import { initializeSignify, 
         initializeAndConnectClient,
         createNewAID,
         addEndRoleForAID,
         generateOOBI,
         resolveOOBI,
         createTimestamp,
         DEFAULT_IDENTIFIER_ARGS,
         DEFAULT_TIMEOUT_MS,
         ROLE_AGENT
       } from './scripts_ts/utils.ts';

const SCHEMA_SERVER_HOST = 'http://vlei-server:7723/';

// Clients setup
// Initialize Issuer, Holder and Verifier CLients, Create AIDs for each one, assign 'agent' role, 
// and generate OOBIs 

// Issuer Client
const issuerBran = randomPasscode()
const issuerAidAlias = 'issuerAid'
const { client: issuerClient } = await initializeAndConnectClient(issuerBran)
const { aid: issuerAid} = await createNewAID(issuerClient, issuerAidAlias, DEFAULT_IDENTIFIER_ARGS);
await addEndRoleForAID(issuerClient, issuerAidAlias, ROLE_AGENT);
const issuerOOBI = await generateOOBI(issuerClient, issuerAidAlias, ROLE_AGENT);

// Holder Client
const holderBran = randomPasscode()
const holderAidAlias = 'holderAid'
const { client: holderClient } = await initializeAndConnectClient(holderBran)
const { aid: holderAid} = await createNewAID(holderClient, holderAidAlias, DEFAULT_IDENTIFIER_ARGS);
await addEndRoleForAID(holderClient, holderAidAlias, ROLE_AGENT);
const holderOOBI = await generateOOBI(holderClient, holderAidAlias, ROLE_AGENT);

// Verifier Client
const verifierBran = randomPasscode()
const verifierAidAlias = 'verifierAid'
const { client: verifierClient } = await initializeAndConnectClient(verifierBran)
const { aid: verifierAid} = await createNewAID(verifierClient, verifierAidAlias, DEFAULT_IDENTIFIER_ARGS);
await addEndRoleForAID(verifierClient, verifierAidAlias, ROLE_AGENT);
const verifierOOBI = await generateOOBI(verifierClient, verifierAidAlias, ROLE_AGENT);

// Clients OOBI Resolution
// Resolve OOBIs to establish connections Issuer-Holder, Holder-Verifier
const issuerContactAlias = 'issuerContact';
const holderContactAlias = 'holderContact';
const verifierContactAlias = 'verifierContact';

await resolveOOBI(issuerClient, holderOOBI, holderContactAlias);
await resolveOOBI(holderClient, issuerOOBI, issuerContactAlias);
await resolveOOBI(verifierClient, holderOOBI, holderContactAlias);
await resolveOOBI(holderClient, verifierOOBI, verifierContactAlias);

// Schemas OOBI Resolution
// Resolve the Schemas from the Schema Server Host
const schemaContactAlias = 'schemaContact';
const schemaSaid = 'EGUPiCVO73M9worPwR3PfThAtC0AJnH5ZgwsXf6TzbVK';
const schemaOOBI = `http://vlei-server:7723/oobi/${schemaSaid}`;

await resolveOOBI(issuerClient, schemaOOBI, schemaContactAlias);
await resolveOOBI(holderClient, schemaOOBI, schemaContactAlias);
await resolveOOBI(verifierClient, schemaOOBI, schemaContactAlias);

console.log("Done.")

Using Passcode (bran): DXznjf4R7IDlqfpmLYdO_
Client boot process initiated with Keria agent.
  Client AID Prefix:  EEeOzX50aWiPxTNLsMFugfnq6rvGTKT92AowJy5nIEKc
  Agent AID Prefix:   EBQmHeC25-ZsZt8cu3ntrRH6eHvdtNBaBy9EQfatooZp
Initiating AID inception for alias: issuerAid
Successfully created AID with prefix: EFRKd21-DKtlOmXKI6UFibk-5ukXfmdc9ohgb7uBnkEi
Assigning 'agent' role to Keria Agent EBQmHeC25-ZsZt8cu3ntrRH6eHvdtNBaBy9EQfatooZp for AID alias issuerAid
Successfully assigned 'agent' role for AID alias issuerAid.
Generating OOBI for AID alias issuerAid with role agent
Generated OOBI URL: http://keria:3902/oobi/EFRKd21-DKtlOmXKI6UFibk-5ukXfmdc9ohgb7uBnkEi/agent/EBQmHeC25-ZsZt8cu3ntrRH6eHvdtNBaBy9EQfatooZp
Using Passcode (bran): CIkc_FPtrgdkFKfu7tZtf
Client boot process initiated with Keria agent.
  Client AID Prefix:  EGSFN0uBsdxQwOImmV1dXph3b8bBICzPAYUTDSjB871P
  Agent AID Prefix:   EFsps3kuZyJS29LcyPi2HyxdUNMIbZ8dcEckhbttFsZl
Initiating AID inception for alias: holderAid
Successfu

In [2]:
//Create Issuer credential Registry

const issuerRegistryName = 'issuerRegistry'

const createRegistryResult = await issuerClient
    .registries()
    .create({ name: issuerAidAlias, registryName: issuerRegistryName });

const createRegistryOperation = await createRegistryResult.op();

const createRegistryResponse = await issuerClient
    .operations()
    .wait(createRegistryOperation, AbortSignal.timeout(DEFAULT_TIMEOUT_MS));

await issuerClient.operations().delete(createRegistryOperation.name);



In [3]:
// Listing Registries

const issuerRegistries = await issuerClient.registries().list(issuerAidAlias);
const issuerRegistry = issuerRegistries[0]
console.log(issuerRegistry)

{
  name: "issuerRegistry",
  regk: "ENj7XJNbjMKyvfMFhi6YkpidpC0CZ6t-XRdD8qif1bhA",
  pre: "EFRKd21-DKtlOmXKI6UFibk-5ukXfmdc9ohgb7uBnkEi",
  state: {
    vn: [ 1, 0 ],
    i: "ENj7XJNbjMKyvfMFhi6YkpidpC0CZ6t-XRdD8qif1bhA",
    s: "0",
    d: "ENj7XJNbjMKyvfMFhi6YkpidpC0CZ6t-XRdD8qif1bhA",
    ii: "EFRKd21-DKtlOmXKI6UFibk-5ukXfmdc9ohgb7uBnkEi",
    dt: "2025-05-16T04:11:53.200139+00:00",
    et: "vcp",
    bt: "0",
    b: [],
    c: [ "NB" ]
  }
}


In [4]:
// Retrieve Schemas

const issuerSchema = await issuerClient.schemas().get(schemaSaid);
console.log(issuerSchema)

// List Schemas

const issuerSchemas = await holderClient.schemas().list();
console.log(issuerSchemas)

{
  "$id": "EGUPiCVO73M9worPwR3PfThAtC0AJnH5ZgwsXf6TzbVK",
  "$schema": "http://json-schema.org/draft-07/schema#",
  title: "EventPass",
  description: "Event Pass Schema",
  type: "object",
  credentialType: "EventPassCred",
  version: "1.0.0",
  properties: {
    v: { description: "Credential Version String", type: "string" },
    d: { description: "Credential SAID", type: "string" },
    u: { description: "One time use nonce", type: "string" },
    i: { description: "Issuer AID", type: "string" },
    ri: { description: "Registry SAID", type: "string" },
    s: { description: "Schema SAID", type: "string" },
    a: {
      oneOf: [
        { description: "Attributes block SAID", type: "string" },
        {
          "$id": "ELppbffpWEM-uufl6qpVTcN6LoZS2A69UN4Ddrtr_JqE",
          description: "Attributes block",
          type: "object",
          properties: [Object],
          additionalProperties: false,
          required: [Array]
        }
      ]
    }
  },
  additionalPropert

In [5]:
const credentialClaims = {
    "eventName":"GLEIF Summit",
    "accessLevel":"staff",
    "validDate":"2026-10-01"
}

const issueResult = await issuerClient
    .credentials()
    .issue(
        issuerAidAlias,
        {
            ri: issuerRegistry.regk,
            s: schemaSaid,
            a: {
                i: holderAid.i,
                ...credentialClaims                
            }
        });

const issueOperation = await issueResult.op;

const issueResponse = await issuerClient
    .operations()
    .wait(issueOperation, AbortSignal.timeout(DEFAULT_TIMEOUT_MS));

const credentialSaid = issueResponse.response.ced.d

In [6]:
// const issuerCredentials = await issuerClient.credentials().list();
// console.log(issuerCredentials)

const issuerCredential = await issuerClient.credentials().get(credentialSaid);
console.log(issuerCredential)

{
  sad: {
    v: "ACDC10JSON0001c4_",
    d: "EERdpjmhPCz4Y9lPRxdFzJf1B15XcHfjDq9IYcvblrSj",
    i: "EFRKd21-DKtlOmXKI6UFibk-5ukXfmdc9ohgb7uBnkEi",
    ri: "ENj7XJNbjMKyvfMFhi6YkpidpC0CZ6t-XRdD8qif1bhA",
    s: "EGUPiCVO73M9worPwR3PfThAtC0AJnH5ZgwsXf6TzbVK",
    a: {
      d: "EKqjk_uTqiKEEhHxN_UrI-DYy1R6-2-08-5m75mocNW7",
      i: "EPXclWGHGuj6H7W2oHjDvkezIUs9qpoPIaNdCQXf7lfu",
      eventName: "GLEIF Summit",
      accessLevel: "staff",
      validDate: "2026-10-01",
      dt: "2025-05-16T04:11:53.450000+00:00"
    }
  },
  atc: "-IABEERdpjmhPCz4Y9lPRxdFzJf1B15XcHfjDq9IYcvblrSj0AAAAAAAAAAAAAAAAAAAAAAAEERdpjmhPCz4Y9lPRxdFzJf1B15XcHfjDq9IYcvblrSj",
  iss: {
    v: "KERI10JSON0000ed_",
    t: "iss",
    d: "EGs6RNJSeMN_cyWrEAvi3gTepRb7ckAqogUruRG1dsoI",
    i: "EERdpjmhPCz4Y9lPRxdFzJf1B15XcHfjDq9IYcvblrSj",
    s: "0",
    ri: "ENj7XJNbjMKyvfMFhi6YkpidpC0CZ6t-XRdD8qif1bhA",
    dt: "2025-05-16T04:11:53.450000+00:00"
  },
  issatc: "-VAS-GAB0AAAAAAAAAAAAAAAAAAAAAACEFEflGchXS6YMj6PxtmUwM

In [20]:

const [grant, gsigs, gend] = await issuerClient.ipex().grant({
    senderName: issuerAidAlias,
    acdc: new Serder(issuerCredential.sad), // ACDC (Credential)
    iss: new Serder(issuerCredential.iss),  // Event (Registry - ACDC)
    anc: new Serder(issuerCredential.anc),  // Event (ACDC - Issuer KEL - ISS Event) 
    ancAttachment: issuerCredential.ancatc, // Credential signatures
    recipient: holderAid.i,
    datetime: createTimestamp(),
});

const submitGrantOperation = await issuerClient
    .ipex()
    .submitGrant(
        issuerAidAlias,
        grant, 
        gsigs, 
        gend,
        [holderAid.i]
    );

const submitGrantResponse = await issuerClient
    .operations()
    .wait(submitGrantOperation, AbortSignal.timeout(DEFAULT_TIMEOUT_MS));

In [21]:
submitGrantResponse

{
  name: "exchange.EDl5NNOCDF2LUkDbUMDNOxsi3_wqV-H63VPwa-0d6VXZ",
  metadata: { said: "EDl5NNOCDF2LUkDbUMDNOxsi3_wqV-H63VPwa-0d6VXZ" },
  done: true,
  error: null,
  response: { said: "EDl5NNOCDF2LUkDbUMDNOxsi3_wqV-H63VPwa-0d6VXZ" }
}

In [ ]:
263